1. Write a MongoDB query to display all the documents in the collection restaurants.

2. Write a MongoDB query to display the fields restaurant_id, name, borough and cuisine for all the documents in the collection restaurant. 

3. Write a MongoDB query to display the fields restaurant_id, name, borough and cuisine, but exclude the field _id for all the documents in the collection restaurant. Go to the editor
Click me to see the solution

4. Write a MongoDB query to display the fields restaurant_id, name, borough and zip code, but exclude the field _id for all the documents in the collection restaurant. Go to the editor
Click me to see the solution

5. Write a MongoDB query to display all the restaurant which is in the borough Bronx. Go to the editor
Click me to see the solution

6. Write a MongoDB query to display the first 5 restaurant which is in the borough Bronx. Go to the editor
Click me to see the solution

7.Write a MongoDB query to display the next 5 restaurants after skipping first 5 which are in the borough Bronx. Go to the editor
Click me to see the solution

8. Write a MongoDB query to find the restaurants who achieved a score more than 90. Go to the editor
Click me to see the solution

9. Write a MongoDB query to find the restaurants that achieved a score, more than 80 but less than 100. Go to the editor
Click me to see the solution

10. Write a MongoDB query to find the restaurants which locate in latitude value less than -95.754168.Go to the editor
Click me to see the solution

11. Write a MongoDB query to find the restaurants that do not prepare any cuisine of 'American' and their grade score more than 70 and latitude less than -65.754168. Go to the editor
Click me to see the solution

12. Write a MongoDB query to find the restaurants which do not prepare any cuisine of 'American' and achieved a score more than 70 and located in the longitude less than -65.754168.
Note : Do this query without using $and operator. Go to the editor
Click me to see the solution

13. Write a MongoDB query to find the restaurants which do not prepare any cuisine of 'American ' and achieved a grade point 'A' not belongs to the borough Brooklyn. The document must be displayed according to the cuisine in descending order. Go to the editor
Click me to see the solution

14. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants which contain 'Wil' as first three letters for its name. Go to the editor
Click me to see the solution

15. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants which contain 'ces' as last three letters for its name. Go to the editor
Click me to see the solution

16. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants which contain 'Reg' as three letters somewhere in its name. Go to the editor
Click me to see the solution

17. Write a MongoDB query to find the restaurants which belong to the borough Bronx and prepared either American or Chinese dish. Go to the editor
Click me to see the solution

18. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants which belong to the borough Staten Island or Queens or Bronxor Brooklyn. Go to the editor
Click me to see the solution

19. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants which are not belonging to the borough Staten Island or Queens or Bronxor Brooklyn. Go to the editor
Click me to see the solution

20. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants which achieved a score which is not more than 10. Go to the editor
Click me to see the solution

21. Write a MongoDB query to find the restaurant Id, name, borough and cuisine for those restaurants which prepared dish except 'American' and 'Chinees' or restaurant's name begins with letter 'Wil'. Go to the editor
Click me to see the solution

22. Write a MongoDB query to find the restaurant Id, name, and grades for those restaurants which achieved a grade of "A" and scored 11 on an ISODate "2014-08-11T00:00:00Z" among many of survey dates.. Go to the editor
Click me to see the solution

23. Write a MongoDB query to find the restaurant Id, name and grades for those restaurants where the 2nd element of grades array contains a grade of "A" and score 9 on an ISODate "2014-08-11T00:00:00Z". Go to the editor
Click me to see the solution

24. Write a MongoDB query to find the restaurant Id, name, address and geographical location for those restaurants where 2nd element of coord array contains a value which is more than 42 and upto 52.. Go to the editor
Click me to see the solution

25. Write a MongoDB query to arrange the name of the restaurants in ascending order along with all the columns. Go to the editor
Click me to see the solution

26. Write a MongoDB query to arrange the name of the restaurants in descending along with all the columns. Go to the editor
Click me to see the solution

27. Write a MongoDB query to arranged the name of the cuisine in ascending order and for that same cuisine borough should be in descending order. Go to the editor
Click me to see the solution

28. Write a MongoDB query to know whether all the addresses contains the street or not. Go to the editor
Click me to see the solution

29. Write a MongoDB query which will select all documents in the restaurants collection where the coord field value is Double. Go to the editor
Click me to see the solution

30. Write a MongoDB query which will select the restaurant Id, name and grades for those restaurants which returns 0 as a remainder after dividing the score by 7. Go to the editor
Click me to see the solution

31. Write a MongoDB query to find the restaurant name, borough, longitude and attitude and cuisine for those restaurants which contains 'mon' as three letters somewhere in its name. Go to the editor
Click me to see the solution

32. Write a MongoDB query to find the restaurant name, borough, longitude and latitude and cuisine for those restaurants which contain 'Mad' as first three letters of its name. Go to the editor